In [ ]:
import pandas as pd
import numpy as np
import bisect
import glob
import math
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
#import networkx as nx
import os
import random
from random import randint
import re
import time
from datetime import date, timedelta, datetime
from scipy import stats
from scipy.stats import ks_2samp
from sklearn.utils.extmath import cartesian
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 300)

In [ ]:
dh = pd.read_csv("D:\\Amir_Nohekhan\\probe_int_map.csv", delimiter=',')
dh

# Input parameters

In [ ]:
start_year = 2018
start_month = 1
start_day = 1
end_year = 2020
end_month = 12
end_day = 31

# Setting up the graph database

In [ ]:
Graph = pd.read_csv("D:\\Amir_Nohekhan\\Graph.csv", delimiter=',' ,dtype=None)
Graph.drop_duplicates(subset=['seg_id'],inplace=True,keep='first',ignore_index=True)
Graph_Structure = pd.read_csv("D:\\Amir_Nohekhan\\Graph_Structure.csv", delimiter=',', 
                              dtype=None)
Graph_Structure["pair"] = "(" + Graph_Structure["Up"].astype(str) + "," +Graph_Structure["Dn"].astype(str) + ")"

In [ ]:
Graph_Structure = Graph_Structure.rename(columns={"Up": "seg_id"})
Graph_Structure = Graph_Structure.merge(Graph,on=["seg_id"], how='left')
Graph_Structure = Graph_Structure.rename(columns={"seg_id": "Up", "roadname":"roadname_up", "Lanes":"lanes_up", 
                                                  "XD":"xd_up", "TMC":"tmc_up"})
Graph_Structure = Graph_Structure.rename(columns={"Dn": "seg_id"})
Graph_Structure = Graph_Structure.merge(Graph,on=["seg_id"], how='left')
Graph_Structure = Graph_Structure.rename(columns={"seg_id": "Dn", "roadname":"roadname_dn", "Lanes":"lanes_dn", 
                                                  "XD":"xd_dn", "TMC":"tmc_dn"})
Graph[['XD','seg_id']] = Graph[['XD','seg_id']].astype(int).astype(str)
Graph_Structure[['xd_up','xd_dn','Up','Dn']] = Graph_Structure[['xd_up','xd_dn','Up','Dn']].astype(int).astype(str)

In [ ]:
l1 = Graph_Structure.pair.unique().tolist()
l2 = []
sdate = date(start_year, start_month, start_day)   # start date
edate = date(end_year, end_month, end_day)   # end date
delta = edate - sdate       # as timedelta
for i in range(delta.days + 1):
    day = sdate + timedelta(days=i)
    l2.append(str(day))
l3 = [ x for x in range(0,24) ]
l4 = [ x for x in range(1,5) ]
df=pd.DataFrame(cartesian((l1, l2, l3, l4)), columns=['pair', 'Date', 'hour','quarter'])
df['Date'] = pd.to_datetime(df.Date)
df['year']=df.Date.dt.year
df['month']=df.Date.dt.month
df['weekday'] = df.Date.dt.weekday
df['day'] = df.Date.dt.day
df[['hour','quarter']] = df[['hour','quarter']].astype('float64')
df["minute"] = (df["quarter"]-1)*15
df['measurement_tstamp'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute' ]])
df = df.drop(['minute'], axis=1)
df = df.merge(Graph_Structure, on=["pair"])
df = df.drop(["Date"], axis=1)
print(df.shape)

# Building the Speed Database

In [ ]:
speed_XD = pd.read_csv("D:\\Amir_Nohekhan\\Speed\\Inrix_XD\\Readings.csv", delimiter=',',
                       dtype=None)
speed_XD['measurement_tstamp'] = pd.to_datetime(speed_XD['measurement_tstamp'])
speed_XD=speed_XD.drop_duplicates(subset=['xd_id','measurement_tstamp'])
speed_XD['xd_id'] = speed_XD['xd_id'].astype(int).astype(str)
speed_TMC = pd.read_csv("D:\\Amir_Nohekhan\\Speed\\Inrix_TMC\\Readings.csv", delimiter=',',
                        dtype=None)
speed_TMC['measurement_tstamp'] = pd.to_datetime(speed_TMC['measurement_tstamp'])
speed_TMC=speed_TMC.drop_duplicates(subset=['tmc_code','measurement_tstamp'])

In [ ]:
speed_XD.dtypes

In [ ]:
dg = df.copy()
df = df.rename(columns={"tmc_up": "tmc_code"})
speed_TMC = speed_TMC.rename(columns={"speed": "speed_up", "average_speed":"avg_speed_up",
                                      "reference_speed":"ref_speed_up"})
df = df.merge(speed_TMC,on=["tmc_code","measurement_tstamp"], how='left')
df = df.rename(columns={"tmc_code":"tmc_up", "tmc_dn": "tmc_code"})
speed_TMC.columns = speed_TMC.columns.str.replace("up", "dn")
df = df.merge(speed_TMC,on=["tmc_code","measurement_tstamp"], how='left')
df = df.rename(columns={"tmc_code": "tmc_dn"})
df = df.sort_values(by=["measurement_tstamp","pair"], ascending=True)
df.head()

In [ ]:
dg = dg.rename(columns={"xd_up": "xd_id"})
speed_XD = speed_XD.rename(columns={"speed": "speed_up", "average_speed":"avg_speed_up","reference_speed":"ref_speed_up"})
dg = dg.merge(speed_XD,on=["xd_id","measurement_tstamp"], how='left')
dg = dg.rename(columns={"xd_id":"xd_up", "xd_dn": "xd_id"})
speed_XD.columns = speed_XD.columns.str.replace("up", "dn")
dg = dg.merge(speed_XD,on=["xd_id","measurement_tstamp"], how='left')
dg = dg.rename(columns={"xd_id": "xd_dn"})
dg = dg.sort_values(by=["measurement_tstamp","pair"], ascending=True)
dg.head()

In [ ]:
df.set_index(['pair','measurement_tstamp'], inplace=True)
df.update(dg.set_index(['pair','measurement_tstamp']))
df=df.sort_values(["pair","measurement_tstamp"])
df = df.reset_index(drop=False)
print(df.shape)
#df = df.fillna(method='ffill') # filling nulls with preceding values

In [ ]:
Counts_df = pd.read_csv("D:\\Amir_Nohekhan\\Camera_Traffic_Counts.csv", delimiter=',',dtype=None)
det_list = Graph_Structure.Detector_ID.unique().tolist()
Counts_df = Counts_df.loc[Counts_df["ATD Device ID"].isin(det_list)]
Counts_df['measurement_tstamp'] = pd.to_datetime(Counts_df[['Year', 'Month', 'Day', 'Hour', 'Minute' ]])
Counts_df = Counts_df.drop(["Record ID","Read Date","Speed StdDev","Seconds in Zone StdDev","Month","Day","Year","Hour",
                            "Minute","Day of Week","Bin Duration (Seconds)"],axis=1).rename(columns={"ATD Device ID":
                                                                                                     "Detector_ID"})
Counts_df["dir"] = ["EB" if x =="EASTBOUND" else "SB" if x =="SOUTHBOUND" else "WB" if x =="WESTBOUND" else
                    "NB" if x =="NORTHBOUND" else "0" for x in Counts_df["Direction"]]
Counts_df["move"] = ["TH" if x =="THRU" else "RT" if x =="RIGHT TURN" else "LT" if x =="LEFT TURN" else
                     0 for x in Counts_df["Movement"]]
Counts_df = Counts_df[Counts_df["dir"]!="0"]
Counts_df = Counts_df[Counts_df["move"]!="0"]
Counts_df["Direction"] = Counts_df["dir"].astype(str) + "_" + Counts_df["move"].astype(str)
Counts_df = Counts_df.drop(["Movement"], axis=1)
#Counts_df = Counts_df.drop(["dir","move","Movement"], axis=1)

Counts_df_h = Counts_df.copy()
Counts_df_h = Counts_df_h.loc[Counts_df_h["Heavy Vehicle"]==True]
Counts_df_h = Counts_df_h.drop(["Heavy Vehicle","Intersection Name","Speed Average (Miles Per Hour)",
                                "Seconds in Zone Average","dir","move"],axis=1).rename(columns={"Volume":"h_veh"})
Counts_df = Counts_df.loc[Counts_df["Heavy Vehicle"]==False]
Counts_df = Counts_df.rename(columns={"Volume":"l_veh"}).drop(["Heavy Vehicle"],axis=1)

In [ ]:
print(Counts_df.shape)
Counts_df = Counts_df.drop_duplicates(subset=['Detector_ID', 'Direction', 'measurement_tstamp'], keep='last')
print(Counts_df.shape)

In [ ]:
print(Counts_df_h.shape)
Counts_df_h = Counts_df_h.drop_duplicates(subset=['Detector_ID', 'Direction', 'measurement_tstamp'], keep='last')
print(Counts_df_h.shape)

In [ ]:
df = df.merge(Counts_df, on=["Detector_ID","Direction","measurement_tstamp"], how='left')
print(df.shape)
df = df.merge(Counts_df_h, on=["Detector_ID","Direction","measurement_tstamp"], how='left')
print(df.shape)

In [ ]:
df = df.drop(["Speed Average (Miles Per Hour)",'Seconds in Zone Average'], axis=1)
df[['dir', 'move']] = df['Direction'].str.split('_', expand=True)

In [ ]:
# df['time_str'] = df["measurement_tstamp"].astype(str)
# list_na = df[df['speed_up'].isna()].time_str.unique().tolist()
# df = df.loc[~df["time_str"].isin(list_na)]
# list_na = df[df['speed_dn'].isna()].time_str.unique().tolist()
# df = df.loc[~df["time_str"].isin(list_na)]

allow = pd.read_csv("D:\\Amir_Nohekhan\\Allowable_Movements.csv", delimiter=',',
                        dtype=None)
df = df.merge(allow, on=['Detector_ID','dir'], how='left')
df.shape

In [ ]:
dg = df.loc[(df['Detector_ID']==6814)]
dh = df.loc[(df['Detector_ID']==6358)]
dg = dg.merge(dh[['pair','measurement_tstamp','l_veh','h_veh']],on=['pair','measurement_tstamp'],how='left')
dg.head()

In [ ]:
dg['l_veh_x'] = dg['l_veh_x'].fillna(dg['l_veh_y'])
dg['h_veh_x'] = dg['h_veh_x'].fillna(dg['h_veh_y'])
dg  = dg.drop(['l_veh_y','h_veh_y'],axis=1).rename(columns={'l_veh_x':'l_veh','h_veh_x':'h_veh'})
df = df.loc[(df['Detector_ID']!=6358)&(df['Detector_ID']!=6814)]
df = df.append(dg)
df.shape

In [ ]:
df = df.sort_values(by=['Detector_ID','measurement_tstamp','pair'])
df = df.rename(columns={'dir':'up_dir'})
df['dn_dir'] = ['NB' if x =="NB_TH" else 'NB' if x =="WB_RT" else 'NB' if x =="EB_LT" else
                'SB' if x =="SB_TH" else 'SB' if x =="EB_RT" else 'SB' if x =="WB_LT" else
                'EB' if x =="EB_TH" else 'EB' if x =="NB_RT" else 'EB' if x =="SB_LT" else 'WB'for x in df["Direction"]]
df.loc[df['h_veh'].isna(),'h_veh']=0
df['tmcount'] = df['l_veh']+df['h_veh']
df.shape

In [ ]:
df = df.merge(df.groupby(['Detector_ID','measurement_tstamp','up_dir']).agg({'tmcount': lambda x: x.sum(skipna=False)}).reset_index().rename(columns={'tmcount':'up_vol'}),on=['Detector_ID','measurement_tstamp','up_dir'],how='left')
df = df.merge(df.groupby(['Detector_ID','measurement_tstamp','dn_dir']).agg({'tmcount': lambda x: x.sum(skipna=False)}).reset_index().rename(columns={'tmcount':'dn_vol'}),on=['Detector_ID','measurement_tstamp','dn_dir'],how='left')

In [ ]:
dc = pd.read_csv("D:\\Amir_Nohekhan\\data_09_28_21.csv", delimiter=',' ,dtype=None)
dc.columns = [str(col) + '_dc' for col in dc.columns]
dc = dc.rename(columns={'pair_dc':'pair','measurement_tstamp_dc':'measurement_tstamp'})
dc['measurement_tstamp'] = pd.to_datetime(dc['measurement_tstamp'])
df = df.merge(dc[['pair','measurement_tstamp','speed_up_dc','avg_speed_up_dc','ref_speed_up_dc','speed_dn_dc',
                  'avg_speed_dn_dc','ref_speed_dn_dc']],on=['pair','measurement_tstamp'],how='left')
df['speed_up'] = df['speed_up'].fillna(df['speed_up_dc'])
df['speed_dn'] = df['speed_dn'].fillna(df['speed_dn_dc'])
df['avg_speed_up'] = df['avg_speed_up'].fillna(df['avg_speed_up_dc'])
df['avg_speed_dn'] = df['avg_speed_dn'].fillna(df['avg_speed_dn_dc'])
df['ref_speed_up'] = df['ref_speed_up'].fillna(df['ref_speed_up_dc'])
df['ref_speed_dn'] = df['ref_speed_dn'].fillna(df['ref_speed_dn_dc'])
df = df.drop(['speed_up_dc','avg_speed_up_dc','ref_speed_up_dc','speed_dn_dc',
              'avg_speed_dn_dc','ref_speed_dn_dc'],axis=1)
df.head()

In [ ]:
dg = df.copy()
dg = df.drop(['pair','Dn','Direction','Exclusive_Lane','roadname_up','xd_up','tmc_up','roadname_dn','xd_dn',
              'tmc_dn','lanes_dn','speed_dn','avg_speed_dn','ref_speed_dn','Intersection Name','l_veh','move',
              'h_veh','movements','dn_dir','tmcount','dn_vol'],axis=1)
dg = dg.rename(columns={'Up':'xd','lanes_up':'lanes','speed_up':'speed','avg_speed_up':'avg_speed',
                        'ref_speed_up':'ref_speed','up_dir':'dir','up_vol':'vol'})
dh = df.copy()
dh = df.drop(['pair','Up','Direction','Exclusive_Lane','roadname_dn','xd_dn','tmc_dn','roadname_up','xd_up',
              'tmc_up','lanes_up','speed_up','avg_speed_up','ref_speed_up','Intersection Name','l_veh','move',
              'h_veh','movements','up_dir','tmcount','up_vol'],axis=1)
dh = dh.rename(columns={'Dn':'xd','lanes_dn':'lanes','speed_dn':'speed','avg_speed_dn':'avg_speed',
                        'ref_speed_dn':'ref_speed','dn_dir':'dir','dn_vol':'vol'})
dg = pd.concat((dg,dh))
dg = dg.drop_duplicates(subset=['measurement_tstamp','xd','Detector_ID'])
dh = dg[['measurement_tstamp','xd','vol']].groupby(['measurement_tstamp','xd']).agg({'vol': lambda x: x.sum(skipna=False)})
dg = dg.drop(['vol'],axis=1).merge(dh,on=['measurement_tstamp','xd'],how='left')
dg.head()

In [ ]:
dg['time'] = dg['hour']+dg['quarter']/4
df['time'] = df['hour']+df['quarter']/4
dh = dg[['xd','time','vol']].groupby(['xd','time']).mean().reset_index()
dh = dh.interpolate()
dh = dh.groupby(['xd'])['vol'].sum().reset_index()
dh = dh.rename(columns={'vol':'aadt'})
dh['aadt'] = dh['aadt'].astype(int)
dg = dg.merge(dh,on=['xd'],how='left')
df = df.merge(dg[['xd','aadt']].rename(columns={'xd':'xd_up'}),on=['xd_up'],how='left').rename(columns={'aadt':'aadt_up'})
df = df.merge(dg[['xd','aadt']].rename(columns={'xd':'xd_dn'}),on=['xd_dn'],how='left').rename(columns={'aadt':'aadt_dn'})

In [ ]:
df = df.merge(dh[['xd','aadt']].rename(columns={'xd':'xd_up'}),on=['xd_up'],how='left').rename(columns={'aadt':'aadt_up'})
df = df.merge(dh[['xd','aadt']].rename(columns={'xd':'xd_dn'}),on=['xd_dn'],how='left').rename(columns={'aadt':'aadt_dn'})

In [ ]:
dg.to_csv("C:\\Users\\szahedi1\\Amir_Thesis\\xd_10_01_21.csv", sep=',',index=False)
df.to_csv("C:\\Users\\szahedi1\\Amir_Thesis\\data_10_01_21.csv", sep=',',index=False)

In [ ]:
dg

# Adding probe tm counts

In [ ]:
df = pd.read_csv("D:\\Amir_Nohekhan\\2020_data_10_26.csv", delimiter=',' ,dtype={'pair':'object',
'measurement_tstamp' : 'object',
'hour' : 'int64',
'quarter' : 'int64',
'year' : 'int64',
'month' : 'int64',
'weekday' : 'int64',
'day' : 'int64',
'Up' : 'int64',
'Dn' : 'int64',
'Detector_ID' : 'int64',
'Direction' : 'object',
'Exclusive_Lane' : 'int64',
'roadname_up' : 'object',
'xd_up' : 'int64',
'tmc_up' : 'object',
'lanes_up' : 'int64',
'roadname_dn' : 'object',
'xd_dn' : 'int64',
'tmc_dn' : 'object',
'lanes_dn' : 'int64',
'speed_up' : 'float64',
'avg_speed_up' : 'float64',
'ref_speed_up' : 'float64',
'speed_dn' : 'float64',
'avg_speed_dn' : 'float64',
'ref_speed_dn' : 'float64',
'Intersection Name' : 'object',
'l_veh' : 'float64',
'up_dir' : 'object',
'move' : 'object',
'h_veh' : 'float64',
'movements' : 'int64',
'dn_dir' : 'object',
'tmcount' : 'float64',
'up_vol' : 'float64',
'dn_vol' : 'float64',
'time' : 'float64',
'aadt_up' : 'int64',
'aadt_dn' : 'int64'
})
db = pd.read_csv("D:\\Amir_Nohekhan\\2020_xd_10_26.csv", delimiter=',' ,dtype={'measurement_tstamp' : 'object',
'hour' : 'int64',
'quarter' : 'int64',
'year' : 'int64',
'month' : 'int64',
'weekday' : 'int64',
'day' : 'int64',
'xd' : 'int64',
'Detector_ID' : 'int64',
'lanes' : 'int64',
'speed' : 'float64',
'avg_speed' : 'float64',
'ref_speed' : 'float64',
'dir' : 'object',
'vol' : 'float64',
'time' : 'float64',
'aadt' : 'int64'})

In [ ]:
dg = pd.read_csv("D:\\Amir_Nohekhan\\probe_tm.csv", delimiter=',')
dh = pd.read_csv("D:\\Amir_Nohekhan\\probe_int_map.csv", delimiter=',')
dh = dh.rename(columns={'probeintname':'intersection_name'})
dg = dg.merge(dh,on=['intersection_name'],how = 'left')
dg = dg.drop(['Unnamed: 0','trip_id','intersection_id','intersection_name','movement_id','movement_name',
              'intersection_offset_m','tt_sec','start_utc_ts','total_stop_duration_sec','movement_type',
              'approach_speed_kph','stop_delay_sec','control_delay_sec','queue_delay_sec','approach_mseq_idx',
              'movement_sequences','points','traj_idx','start_segment_idx','end_segment_idx','start_segment_offset_m',
              'end_segment_offset_m','waypoint_freq_sec','ref_tt_sec','turn_group_id','split_failure_count','intname',
             'movement_turn_maneuver','movement_dir'],
             axis=1)
dg["date"] = dg['start_date_local'].str.split("T", expand=True)[0]
dg["time"] = dg['start_date_local'].str.split("T", expand=True)[1]
dg["time"] = dg['time'].str.split("-", expand=True)[0]
dg['measurement_tstamp'] = pd.to_datetime(dg['date'] + ' ' + dg['time'])
dg['measurement_tstamp'] = dg['measurement_tstamp'].dt.floor('15T')
dg = dg.rename(columns={'movement_inbound_dir':'dir_up','movement_outbound_dir':'dir_dn'})
df['measurement_tstamp'] = pd.to_datetime(df['measurement_tstamp'])
db['measurement_tstamp'] = pd.to_datetime(db['measurement_tstamp'])
dq = dg.groupby(['Detector_ID','measurement_tstamp','dir_up','dir_dn']).count().reset_index().drop(['date','time'],axis=1)
dq = dq.rename(columns={'start_date_local':'p_tmcount'})
dq['dir_up'] = dq['dir_up'] + 'B'
dq['dir_dn'] = dq['dir_dn'] + 'B'

In [ ]:
df = df.merge(dq,on=['Detector_ID','measurement_tstamp','dir_up','dir_dn'],how='left')
df['p_tmcount'] = df['p_tmcount'].fillna(value=0)
dh = df[['measurement_tstamp','Detector_ID','dir_up',
         'p_tmcount']].groupby(['measurement_tstamp','dir_up',
                                'Detector_ID']).agg({'p_tmcount': lambda x: x.sum(skipna=False)}).reset_index().rename(columns={'p_tmcount':'p_vol_up'})
df = df.merge(dh,on=['measurement_tstamp','Detector_ID','dir_up'],how='left')

dh = df[['measurement_tstamp','Detector_ID','dir_dn',
         'p_tmcount']].groupby(['measurement_tstamp','dir_dn',
                                'Detector_ID']).agg({'p_tmcount': lambda x: x.sum(skipna=False)}).reset_index().rename(columns={'p_tmcount':'p_vol_dn'})
df = df.merge(dh,on=['measurement_tstamp','Detector_ID','dir_dn'],how='left')
df.head()

In [ ]:
df[df['p_vol_dn'].isna()]

In [ ]:
df.shape

In [ ]:
df.to_csv("D:\\Amir_Nohekhan\\2020_data_11_22.csv", sep=',',index=False)

In [ ]:
df = pd.read_csv("D:\\Amir_Nohekhan\\2020_data_11_22.csv", delimiter=',')
df.head()

In [ ]:
db = pd.read_csv("D:\\Amir_Nohekhan\\2020_xd_10_26.csv", delimiter=',')
db.head()

In [ ]:
dg = df.copy()
dg = dg.rename(columns={'xd_up':'xd','p_vol_up':'p_vol'})
db = db.merge(dg[['measurement_tstamp','xd','p_vol']],on=['measurement_tstamp','xd'],how='left')

dg = df.copy()
dg = dg.rename(columns={'xd_dn':'xd','p_vol_dn':'p_vol'})
db = db.merge(dg[['measurement_tstamp','xd','p_vol']],on=['measurement_tstamp','xd'],how='left')
db.p_vol_x = db.p_vol_x.fillna(db.p_vol_y)
db = db.rename(columns={'p_vol_x':'p_vol'}).drop(['p_vol_y'],axis=1)

In [ ]:
db

In [ ]:
db[db['p_vol'].isna()]

In [ ]:
df.loc[(df['measurement_tstamp']=='2020-01-01 00:00:00')&(df['Detector_ID']==6192)]

In [ ]:
db[db['p_vol'].isna()]

In [ ]:
db.to_csv("D:\\Amir_Nohekhan\\2020_xd_11_22.csv", sep=',',index=False)